In [1]:
from models import HF_LLMQuery, OpenAI_LLMQuery
import openai

# The Easiest way to generate SQL query from text with zero-shot

#### Using HF Model for SQL Queries

First we should specify the configs paths. By default there are some, you can use them at first

In [4]:
MODEL_CONFIG = "configs/models/hf_model_config.yaml"
PROMPT_CONFIG = "configs/prompts/Zero_shot.yaml"

Then we simply query with our question

In [5]:
LLM_Simple = HF_LLMQuery(MODEL_CONFIG, PROMPT_CONFIG)

query = "How many active agency customers did we have on January 1st, 2022?"

ans = LLM_Simple.query(query)
print(ans)

0 active agency customers on january 1st, 2022


#### Using ChatGPT for SQL Queries

Now we specify configs for ChatGPT. Default could be used as well, but it is crucial to provide openai token and write it to the model configs as follows:

```yaml
token: sk1824...
```

In [2]:
MODEL_CONFIG = "configs/models/openai_model_config.yaml"
PROMPT_CONFIG = "configs/prompts/Zero_shot.yaml"

Then we can do exatly the same thing as it was before. 

Sometimes there could be problems with OpenAI API so the token could be invalid.

In [3]:
ChatGPT = OpenAI_LLMQuery(MODEL_CONFIG, PROMPT_CONFIG)
query = "How many active agency customers did we have on January 1st, 2022?"

try:
    ans = ChatGPT.query(query)
    print(ans)
except (openai.error.AuthenticationError, openai.error.RateLimitError) as e:
    print("Invalid Token")

Invalid Token


### Using one-shot

To enable one-shot inference we need to change configs, so the text for few-shot is present there.

In every config one can find a field named "few_shot_text". If the text is passed in it, it will be used as a few_shot. 

In the example before we will use another configs, that has few-shot text like it is shown below.


```yaml
  few_shot_text: "question: get people name with age equal 25 table: id, name, age \n SELECT name FROM table WHERE age = 25"
```

In [4]:
MODEL_CONFIG = (
    "configs/models/hf_model_config.yaml"  # here we could have used OpenAI model
)
PROMPT_CONFIG = "configs/prompts/One_shot.yaml"

In [5]:
LLM_Simple = HF_LLMQuery(MODEL_CONFIG, PROMPT_CONFIG)

query = "How many active agency customers did we have on January 1st, 2022?"

ans = LLM_Simple.query(query)
print(ans)

SELECT COUNT active agency FROM table WHERE date = january 1st, 2022


## Providing table and columns names in a prompt

It is usually a great technique to boost models quality. A lot of HuggingFace models as well expect to see those names to work better.

To do so, there is Prompt Schema that works with this case. To use it, we will need to specify it in configs in a very simply manner as shown below.

```yaml
    PromptClass: QuestionTableRowsPrompt
```

and if we want to have the standart prompting, it should be 

```yaml
    PromptClass: SimplePrompt
```

## Adding model instructions

Notice that we can pass a parameter of instruction, that will serve as "system" message. That could possibly increase generation quality for Large models with emergent abilities.

To pass the instruction we need to change config as well and add a text for an instruction

```yaml
    prompt_configs:
        instruction_text: "Act as a professional SQL developer and answer a question with a step by step reasoning"
```

## Using Question Decomposition to improve performance

As shown in an [article](https://arxiv.org/pdf/2305.14215.pdf) decomposition of the question could help the model to solve more complex task. To do so, lets change our one-shot example to suit the suggested methodology.

Now our one-shot prompt is

```yaml
    prompt_configs:
          few_shot_text: |
            question: get people name with age equal 25 table: id, name, age
            First need to select all the people from table
            SELECT * FROM table
            Second need to select only people with age equal 25, that corresponds to the column age
            SELECT * FROM table WHERE age = 25
            A: SELECT * FROM table WHERE age = 25
```

#### Taking everything together 

We now will have the following config

``` yaml
PromptClass: QuestionTableColumnsPrompt
prompt_configs:
  few_shot_text: |
    question: get people name with age equal 25 table: id, name, age
    First need to select all the people from table
    SELECT * FROM table
    Second need to select only people with age equal 25, that corresponds to the column age
    SELECT * FROM table WHERE age = 25
    A: SELECT * FROM table WHERE age = 25
  instruction_text: "Act as a professional SQL developer and answer a question with a step by step reasoning"
```


In [3]:
MODEL_CONFIG = (
    "configs/models/hf_model_config.yaml"  # here we could have used OpenAI model
)
PROMPT_CONFIG = "configs/prompts/QDecomp.yaml"

LLM_QDecomp = HF_LLMQuery(MODEL_CONFIG, PROMPT_CONFIG)

query = "How many active agency customers did we have on January 1st, 2022?"
tables = "my_table"
columns = ["id", "customer_action", "date"]

ans = LLM_QDecomp.query(query, tables=tables, columns=columns)
print(ans)

A: SELECT COUNT customer_action FROM table WHERE date = january 1st, 2022


## Advanced Usage


#### Changing Models

Changing models is a very simply process. We should just change the variable inside the config, that downloads it from HuggingFace and the class name that it uses.

For example, we would like to take just GPT-2:

```yaml
TokenizerClass: GPT2Tokenizer
ModelClass: GPT2Model

model:
  pretrained_model_name_or_path: gpt2
tokenizer:
  pretrained_model_name_or_path: gpt2
```

#### Changing models generation arguments

The same happens with the generation arguments. One can change the config file and add any relevant arguments for generation.

```yaml
    generation_args:
    max_new_tokens: 32
    num_beams: 12
```


#### Custom Prompt Schemas


There could be cases when user would like to create his own type of prompt, for example, it will contain specification of database as input, or something else.

Therefore, to make new prompt style, one should visit ```prompting/hf_prompt_schemas.py``` or ```prompting/openai_prompt_schemas.py```. THere are differencies due to the API specifics. Then, one should simply implement custom Prompt Schema with a method ```__call_``` and that inherits ```SimplePrompt``` and uses ```_transforms``` method from super class before return. 

For example, we want to pass the name of database

```python

class DB_NamePrompt(SimplePrompt):
    def __call__(self, query, db_name, **kwargs):
        updated_query = "{}, dbname: {}".format(query, db_name)
        return self._transforms(updated_query)
```

Thats it! 

Now if we would like to use this prompt, we simply should change our config to the following:

```yaml
    PromptClass: DB_NamePrompt
```


#### Query and execute the same time

There may be a need to test the functionality of model, so we would like to execute query immediately when the model output was obtained, for example when calculating metrics for Text2SQL Task. To do so we should provide path to the database credentials and after call the ```query_and_execute()``` method just the same way we did before with just query.

Notice, that the time of execution is limited, you can manually change it. 